In [1]:
"""单隐藏层多层感知机"""
import torch
from torch import nn# net[0]     net[1]     net[2]
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.1454],
        [-0.1708]], grad_fn=<AddmmBackward0>)

In [2]:
print(net[2].state_dict())
# 权重是状态 

OrderedDict([('weight', tensor([[-0.1410, -0.2588, -0.2541,  0.0769, -0.2056,  0.0298,  0.2280,  0.2993]])), ('bias', tensor([-0.3221]))])


In [4]:
print(type(net[2].bias))# 最后一层的偏移
print(net[2].bias)# 
print(net[2].bias.data)# .data来访问值   因为它还有一个梯度
print(net[2].bias.grad)# grad为none


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3221], requires_grad=True)
tensor([-0.3221])
None


In [5]:
net[2].bias.grad==None

True

In [6]:
print(*[(name,param.shape)for name,param in net[0].named_parameters()])# 把第一层参数拿出来
print(*[(name,param.shape)for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([-0.3221])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3217],
        [0.3215]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
def init_normal(m):# 正态分布 m是module
    if type(m)==nn.Linear:# 如果module是线性类
        nn.init.normal_(m.weight,mean=0,std=0.01)# 如果是全连接层
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0015,  0.0110, -0.0063, -0.0043]), tensor(0.))

In [12]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [14]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2779, -0.2094,  0.4230,  0.2002])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
def my_init(m):
    if type(m)==nn.Linear:
        print(
            "Init",
            *[(name,param.shape)for name,param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data*=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.8064, -0.0000, -8.1057,  7.8886],
        [-0.0000,  7.5165, -5.5297,  5.4905]], grad_fn=<SliceBackward0>)

In [17]:
net[0].weight.data[:]+=1# 所有元素都加1
net[0].weight.data[0,0]=42# 第一个元素等于42
net[0].weight.data[0]# 第一行

tensor([42.0000,  1.0000, -7.1057,  8.8886])

In [19]:
#参数绑定
shared=nn.Linear(8,8)
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))# 第2个全连接层和第3个隐藏层是share权重
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100  # 同步更新
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
